In [53]:

!sudo apt install sqlite3
!cp $SQLITE_STORE_DIR/faiss_store.db .
!ls -lh faiss_store.db

counter = !cat counter.txt
counter = int(counter[0]) - 1
%env MEMORY_BANK_NAME=bank_pdf_paper_{counter}


Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
sqlite3 is already the newest version (3.37.2-2ubuntu0.3).
0 upgraded, 0 newly installed, 0 to remove and 29 not upgraded.
-rw-r--r-- 1 ubuntu ubuntu 70M Dec  7 06:10 faiss_store.db
env: MEMORY_BANK_NAME=bank_pdf_paper_24


In [54]:
%%bash
echo "select value from kvstore where key = 'faiss_index:v1::${MEMORY_BANK_NAME}';" \
  | sqlite3 faiss_store.db \
  > faiss_index.json


In [55]:
!ls -lh faiss_index.json

-rw-rw-r-- 1 ubuntu ubuntu 13M Dec  7 06:10 faiss_index.json


In [56]:
import json
from IPython.display import display

# Load into dict
faiss_index = json.load(open('faiss_index.json'))
display(faiss_index.keys())
display(len(faiss_index['id_by_index']))
display(len(faiss_index['chunk_by_index']))
display(len(faiss_index['faiss_index']))

# 252 so mostly all contexts are a single chunk

dict_keys(['id_by_index', 'chunk_by_index', 'faiss_index'])

252

252

12903900

In [61]:
import os
from llama_stack_client import LlamaStackClient

# Test run
memory_bank_name = os.getenv("MEMORY_BANK_NAME")

client = LlamaStackClient(
    base_url="http://localhost:5001",
)

response = client.memory.query(
    bank_id=memory_bank_name,
    query=["Hello World"],
    params={"max_chunks": 10},
)

display(response)

for chunk, score in zip(response.chunks, response.scores):
    print(chunk)
    print(score)


QueryDocumentsResponse(chunks=[Chunk(content='The Bey Hive is the name given to Beyoncé\'s fan base. Fans were previously titled "The Beyontourage", (a portmanteau of Beyoncé and entourage). The name Bey Hive derives from the word beehive, purposely misspelled to resemble her first name, and was penned by fans after petitions on the online social networking service Twitter and online news reports during competitions.', document_id='8', token_count=80), Chunk(content="Established in 1988, Chengdu Hi-tech Industrial Development Zone (Chinese: 成都高新技术产业开发区; pinyin: Chéngdū Gāoxīn Jìshù Chǎnyè Kāifā Qū) was approved as one of the first national hi-tech development zones in 1991. In 2000, it was open to APEC and has been recognized as a national advanced hi-tech development zone in successive assessment activities held by China's Ministry of Science and Technology. It ranks 5th among the 53 national hi-tech development zones in China in terms of comprehensive strength.", document_id='184', t

Chunk(content='The Bey Hive is the name given to Beyoncé\'s fan base. Fans were previously titled "The Beyontourage", (a portmanteau of Beyoncé and entourage). The name Bey Hive derives from the word beehive, purposely misspelled to resemble her first name, and was penned by fans after petitions on the online social networking service Twitter and online news reports during competitions.', document_id='8', token_count=80)
0.5918109683380094
Chunk(content="Established in 1988, Chengdu Hi-tech Industrial Development Zone (Chinese: 成都高新技术产业开发区; pinyin: Chéngdū Gāoxīn Jìshù Chǎnyè Kāifā Qū) was approved as one of the first national hi-tech development zones in 1991. In 2000, it was open to APEC and has been recognized as a national advanced hi-tech development zone in successive assessment activities held by China's Ministry of Science and Technology. It ranks 5th among the 53 national hi-tech development zones in China in terms of comprehensive strength.", document_id='184', token_count=13

In [69]:
import pandas as pd

df = pd.read_parquet("data/squad_250.parquet")
display(df.iloc[67])
df.head()

context      Short Films, Big Ideas was launched at the 201...
questions    [What was the name of the project introduced b...
answers      [Short Films, Big Ideas, cinelan, water vapor ...
Name: 67, dtype: object

,context,questions,answers
0,"Sometimes, poly-sided matches that pit every o...","[What happens in a fatal four-way?, In a fatal...","[four wrestlers, each for themselves, fight in..."
1,"In Lebanon, a part of the Christian population...",[Lebanese people of what religion sometimes co...,"[Christian, Arabic, Lebanese Arabic, Arabic, L..."
2,"The current ""Precentor"" (Head of Music) is Tim...","[What term is given to the Head of Music?, Wha...","[""Precentor"", didgeridoo, Tim Johnson, eight]"
3,Israeli universities are among 100 top world u...,[Israeli universities rank where in mathematic...,"[100, six, stem-cell research]"
4,"The current Chief of the Defence Staff, the pr...",[What is the name of the person who is the cur...,"[General Sir Nicholas Houghton, Chief of the D..."


In [73]:
doc_ids = []
questions = []
for _, row in df.iterrows():
    for question in row["questions"]:
        questions.append(question)
        doc_ids.append(_)

print(len(questions))
print(len(doc_ids))
display(questions[:5])
doc_ids[:5]

1174
1174


['What happens in a fatal four-way?',
 'In a fatal four-way, who can make tags?',
 'What was the most common type of match before the fatal four-way? ',
 'Why are tagging rules made?',
 'Lebanese people of what religion sometimes consider Lebanese to be a distinct language?']

[0, 0, 0, 0, 1]

In [85]:
from tqdm import tqdm

top_k = 10

results = []

for i in tqdm(range(0, len(questions))):
    response = client.memory.query(
        bank_id=memory_bank_name,
        query=[questions[i]],
        params={"max_chunks": top_k},
    )
    res = [int(chunk.document_id) for chunk in response.chunks]
    results.append(res)

results

  1%|▌                                                                              | 9/1174 [00:00<00:12, 89.85it/s]

100%|████████████████████████████████████████████████████████████████████████████| 1174/1174 [01:17<00:00, 15.07it/s]


[[0, 89, 177, 216, 222, 239, 99, 158, 170, 112],
 [0, 42, 248, 177, 190, 239, 58, 148, 198, 50],
 [0, 143, 213, 120, 9, 99, 134, 12, 170, 231],
 [0, 42, 190, 58, 239, 148, 248, 46, 147, 1],
 [1, 247, 231, 5, 80, 208, 240, 248, 22, 147],
 [1, 42, 80, 248, 102, 247, 5, 74, 208, 190],
 [1, 231, 80, 247, 86, 102, 248, 53, 221, 214],
 [1, 231, 247, 5, 80, 141, 208, 214, 147, 53],
 [1, 248, 80, 102, 53, 86, 231, 22, 247, 214],
 [2, 152, 227, 230, 114, 59, 237, 8, 42, 137],
 [2, 114, 227, 230, 102, 144, 147, 234, 63, 64],
 [2, 13, 87, 225, 4, 194, 164, 48, 29, 34],
 [2, 84, 235, 167, 133, 21, 127, 90, 32, 35],
 [3, 208, 212, 29, 144, 22, 149, 244, 142, 184],
 [3, 87, 67, 22, 142, 144, 40, 184, 123, 54],
 [3, 22, 47, 232, 201, 142, 123, 1, 67, 226],
 [4, 50, 194, 229, 76, 13, 188, 120, 246, 34],
 [4, 50, 76, 194, 221, 39, 246, 13, 229, 188],
 [4, 50, 194, 245, 76, 13, 229, 188, 25, 221],
 [4, 194, 50, 229, 76, 188, 246, 39, 25, 120],
 [5, 230, 147, 247, 210, 1, 144, 105, 150, 208],
 [5, 230, 1

In [86]:
import numpy as np

res = np.array(results)
res.shape

(1174, 10)

In [94]:
targets = np.array(doc_ids)
display(targets.shape)
targets = np.expand_dims(targets, axis=1)
display(targets.shape)
targets = targets == res
display(targets.shape)
targets = targets.astype(float)
display(targets)

(1174,)

(1174, 1)

(1174, 10)

array([[1., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       ...,
       [1., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.]])

In [95]:
temp = np.sum(targets, axis=1)
temp = np.divide(temp, top_k)
precision = np.sum(temp) / len(temp)
precision

np.float64(0.10042589437819419)

In [96]:
targets = np.logical_or.reduce(targets, axis=1)
targets = targets.astype(float)
recall = np.sum(targets) / len(targets)
recall

np.float64(0.9965928449744463)